In [3]:
#import pybedtools

import numpy as np
import pandas as pd

#import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns
#import seaborn.objects as so

from scipy.stats import pearsonr, spearmanr
#import glob

# Data

In [14]:
#path = "/sybig/projects/GeneRegulation/data/jme/Bachelorarbeit/data/Single_TFBS/All_TFBS.csv"
path = r"C:\Users\julia\MyPython\BA\All_TFBS.csv"
df_combined = pd.read_csv(path)
df_combined = df_combined[df_combined["chr"] != "chrM"]#df_red = reduce_homotypic_tfbs_occurence_consider_strand(df_combined)

In [34]:
def transform_expr_to_log_zero_to_nan(df, expr_iloc_start=9, expr_iloc_end=None):
    if expr_iloc_end is None:
        expr_iloc_end = len(df.columns)
    expr = df.iloc[:,expr_iloc_start:expr_iloc_end]
    replace_zero = expr.replace(0, np.NAN)
    expr_log2 = np.log2(replace_zero)
    df_new = df.copy()
    df_new.iloc[:,expr_iloc_start:expr_iloc_end] = expr_log2
    return df_new

def flatten_tissues_expand_df(tfbs_df, tissue_iloc =9):
    tfbs_part_repeat = tfbs_df.iloc[:,:tissue_iloc]
    tfbs_part_flatten = tfbs_df.iloc[:,tissue_iloc:]
    
    repeat_df =  pd.DataFrame(np.repeat(tfbs_part_repeat.values, tfbs_part_flatten.shape[1], axis=0))
    repeat_df = repeat_df.convert_dtypes(convert_integer = True)
    
    
    flatten_arr = tfbs_part_flatten.to_numpy().flatten()

    tissue_repeat = np.tile(np.array(tfbs_part_flatten.columns), len(tfbs_part_repeat))
    
    repeat_df.columns = tfbs_part_repeat.columns
    repeat_df["GeneExpr"] = flatten_arr
    repeat_df["Tissue"] = tissue_repeat
    
    return repeat_df

def get_mean_expression_per_tf(df_flat, x="log2_GeneExpr"):
    return df_flat.groupby(["tf"])[x].mean()

In [18]:
# Transform all GeneExpression values to log2(GeneExpr)
df_log = transform_expr_to_log_zero_to_nan(df_combined)

In [48]:
# Function to add a new column labeling the Stran-Orientation for the TFBSs

def add_strand_orientation_pct(df):
    """
    Calculates the fraction for TFBS occurences for each TFBS and adds a new column at the end of the DataFrame.
    1 = all non-Template Strand 
    -1 =  all Template strand
    """
    df_change = df.copy()
    df_change["strand_orientation"] = np.array([1 if i == "nT" else -1 for i in df["strand_orientation"]])
    mean_orientation_arr = df_change.groupby(by=["tf", "geneID", "homotypic_count"])["strand_orientation"].transform("mean")
    df_strand = df.copy()
    df_strand.insert(loc=6, column="homotypic_strand_orientation", value=mean_orientation_arr)
    return df_strand

In [50]:
df_log_strand = add_strand_orientation_pct(df_log)

In [52]:
df_log_strand

,chr,geneID,tf,close_tss,dist_tss,strand_orientation,homotypic_strand_orientation,homotypic_count,all_tfbs_count,all_tfbs_unique_count,...,Skin - Not Sun Exposed (Suprapubic),Skin - Sun Exposed (Lower leg),Small Intestine - Terminal Ileum,Spleen,Stomach,Testis,Thyroid,Uterus,Vagina,Whole Blood
0,chr1,ENSG00000163209.14,HMBOX1,159,169,nT,1.0,1,24,20,...,0.164786,-0.048412,0.722466,1.029276,0.977463,1.071763,0.349365,1.387363,12.065248,-1.049905
1,chr1,ENSG00000143569.18,HMBOX1,9,19,T,-1.0,1,12,11,...,5.635609,5.739038,5.148527,5.299391,4.719183,6.031439,5.907852,6.035514,5.787250,4.108524
2,chr10,ENSG00000213390.10,HMBOX1,47,57,nT,1.0,1,73,55,...,3.282885,3.192825,3.046666,4.332708,2.172488,5.034744,2.763836,3.455492,3.135042,2.953265
3,chr1,ENSG00000169241.17,HMBOX1,160,170,nT,1.0,1,115,81,...,5.381802,5.335569,5.551208,5.715893,5.047015,4.516015,5.541948,4.922911,5.015248,4.200065
4,chr1,ENSG00000185499.16,HMBOX1,173,183,nT,1.0,1,23,20,...,2.588565,2.746528,3.217231,1.586885,8.501837,5.176323,4.838952,4.423242,4.512859,-0.736966
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700911,chr7,ENSG00000105851.10,STAT5B,-3,8,T,-1.0,1,33,32,...,-1.994241,-1.836501,1.165430,2.533563,-0.800877,2.312084,-0.187707,-0.720232,-1.207561,2.907275
700912,chr7,ENSG00000075790.10,STAT5B,124,135,nT,1.0,1,31,26,...,3.628190,3.583760,3.162371,3.263185,3.037382,4.919340,5.226123,4.866908,4.450881,1.025029
700913,chr7,ENSG00000128590.4,STAT5B,54,69,T,-1.0,1,71,59,...,4.300490,4.084915,4.377818,4.653060,4.618239,5.744430,5.419539,5.175325,4.564988,3.105678
700914,chr7,ENSG00000198839.9,STAT5B,56,71,nT,1.0,1,122,89,...,3.350497,3.211635,2.663345,2.647545,2.904966,2.831675,3.306262,3.350497,3.389567,1.624335


# FLAT

In [42]:
# Flatten Tissue for all 
df_log_flat = flatten_tissues_expand_df(df_log)

# Rest

In [13]:
def add_strand_orientation_pct(df):
    """
    Calculates the fraction for TFBS occurences for each TFBS and adds a new column at the end of the DataFrame.
    1 = all non-Template Strand 
    -1 =  all Template strand
    """
    df_change = df.copy()
    df_change["strand_orientation"] = np.array([1 if i == "nT" else -1 for i in df["strand_orientation"]])
    mean_orientation_arr = df_change.groupby(by=["tf", "geneID", "homotypic_count"])["strand_orientation"].transform("mean")
    df_strand = df.copy()
    df_strand.insert(loc=6, column="homotypic_strand_orientation", value=mean_orientation_arr)
    return df_strand

In [15]:
df_strand = add_strand_orientation_pct(df_combined)

In [17]:
df_strand

,chr,geneID,tf,close_tss,dist_tss,strand_orientation,homotypic_strand_orientation,homotypic_count,all_tfbs_count,all_tfbs_unique_count,...,Skin - Not Sun Exposed (Suprapubic),Skin - Sun Exposed (Lower leg),Small Intestine - Terminal Ileum,Spleen,Stomach,Testis,Thyroid,Uterus,Vagina,Whole Blood
0,chr1,ENSG00000163209.14,HMBOX1,159,169,nT,1.0,1,24,20,...,1.121,0.967,1.650,2.041,1.969,2.102,1.274,2.616,4285.500,0.483
1,chr1,ENSG00000143569.18,HMBOX1,9,19,T,-1.0,1,12,11,...,49.715,53.410,35.470,39.380,26.340,65.410,60.040,65.595,55.225,17.250
2,chr10,ENSG00000213390.10,HMBOX1,47,57,nT,1.0,1,73,55,...,9.733,9.144,8.263,20.150,4.508,32.780,6.792,10.970,8.785,7.745
3,chr1,ENSG00000169241.17,HMBOX1,160,170,nT,1.0,1,115,81,...,41.695,40.380,46.890,52.560,33.060,22.880,46.590,30.335,32.340,18.380
4,chr1,ENSG00000185499.16,HMBOX1,173,183,nT,1.0,1,23,20,...,6.015,6.711,9.300,3.004,362.500,36.160,28.620,21.455,22.830,0.600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700911,chr7,ENSG00000105851.10,STAT5B,-3,8,T,-1.0,1,33,32,...,0.251,0.280,2.243,5.790,0.574,4.966,0.878,0.607,0.433,7.502
700912,chr7,ENSG00000075790.10,STAT5B,124,135,nT,1.0,1,31,26,...,12.365,11.990,8.953,9.601,8.210,30.260,37.430,29.180,21.870,2.035
700913,chr7,ENSG00000128590.4,STAT5B,54,69,T,-1.0,1,71,59,...,19.705,16.970,20.790,25.160,24.560,53.610,42.800,36.135,23.670,8.608
700914,chr7,ENSG00000198839.9,STAT5B,56,71,nT,1.0,1,122,89,...,10.200,9.264,6.335,6.266,7.490,7.119,9.892,10.200,10.480,3.083


## Pearson correlation of homotypic strand percent against Geneexpression

In [6]:
def refine_pearson_df(df):
    # Which TFBS have only one TFBS per Promotor?
    na_r = df.Pearson_r.isna().to_numpy()
    filterd_constant = df[na_r].TFs.to_numpy()
    print(f"{len(filterd_constant)} were filtered because they have  a constant homotypic count.")

    pearson_df_sort = df.reindex(df["Pearson_r"].abs().sort_values(ascending=False).index).dropna()
    pearson_df_sort_pval = pearson_df_sort[pearson_df_sort["Pearson_p"] < 0.05]
    pearson_df_sort_pval["Pearson_p"] = -np.log10(pearson_df_sort_pval["Pearson_p"])

    # TFBS filterd out through p_value
    filtered_tfbs = pearson_df_sort[pearson_df_sort["Pearson_p"] >= 0.05]
    filtered_by_p = filtered_tfbs.TFs.to_numpy()
    print(f"{len(filtered_by_p)} were filtered because they have  a p-value < 0.05.")

    return pearson_df_sort_pval, filterd_constant, filtered_by_p

### Flat

In [8]:
def flatten_tissues_expand_df(tfbs_df, tissue_iloc =9):
    tfbs_part_repeat = tfbs_df.iloc[:,:tissue_iloc]
    tfbs_part_flatten = tfbs_df.iloc[:,tissue_iloc:]
    
    repeat_df =  pd.DataFrame(np.repeat(tfbs_part_repeat.values, tfbs_part_flatten.shape[1], axis=0))
    flatten_arr = tfbs_part_flatten.to_numpy().flatten()

    tissue_repeat = np.tile(np.array(tfbs_part_flatten.columns), len(tfbs_part_repeat))
    
    repeat_df.columns = tfbs_part_repeat.columns
    repeat_df["GeneExpr"] = flatten_arr
    repeat_df["Tissue"] = tissue_repeat
    
    return repeat_df


def get_pearson(df, x="homotypic_count", tissue="GeneExpr"):
    tfs = df["tf"].unique()
    r_lst = []
    p_lst = []
    tf_lst = []
    for tf in tfs:
        df_tf = df[df["tf"] == tf]
        if len(df_tf) >= 2:
            r, p_value = pearsonr(df_tf[x], np.array(df_tf[tissue]))
            r_lst.append(r)
            p_lst.append(p_value)
            tf_lst.append(tf)
        else:
            print(f"{tf} has less then 2 entrys. No Correlation could be calculated")
    pearson_df = pd.DataFrame({"TFs":tf_lst, "Pearson_r":r_lst, "Pearson_p":p_lst})

    return pearson_df

In [9]:
def reduce_homotypic_tfbs_occurence_consider_strand(df):
    occur_region = df.groupby(["chr", "geneID", "tf", "homotypic_count", "Tissue"]).cumcount()
    df_red = df[occur_region == 0]
    return df_red

In [10]:
flat_strand_df = flatten_tissues_expand_df(df_strand, tissue_iloc=10)
flat_strand_df_red = reduce_homotypic_tfbs_occurence_consider_strand(flat_strand_df)

### Homotypic strand orientation VS. Flat-GeneExpr

In [15]:
pearson_strand_df = get_pearson(flat_strand_df_red, x="homotypic_strand_orientation")

/sybig/home/jme/Bachelorarbeit/.venv/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


In [19]:
pearson_strand_df.to_csv("/sybig/projects/GeneRegulation/data/jme/Bachelorarbeit/data/Single_TFBS/strand_pct_Pearson.csv", index=False)

In [20]:
pearson_df_sort_pval, filterd_constant, filtered_by_p = refine_pearson_df(pearson_strand_df)

1 were filtered because they have  a constant homotypic count.
66 were filtered because they have  a p-value < 0.05.


/tmp/ipykernel_2113880/129602368.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pearson_df_sort_pval["Pearson_p"] = -np.log10(pearson_df_sort_pval["Pearson_p"])


In [23]:
pearson_df_sort_pval

,TFs,Pearson_r,Pearson_p
233,ARNTL,0.256436,79.506366
236,POU2F1,0.214196,6.296585
26,SOX10,-0.189905,3.229528
121,RFX2,0.185116,17.791833
237,ISL1,0.168632,11.764249
...,...,...,...
154,NFE2L2,-0.004393,1.347982
144,REST,-0.004168,1.675244
189,RUNX1,-0.003471,1.605991
145,ESR1,0.003352,1.731789


#### Plot

In [13]:
fig, ax = plt.subplots(figsize=(10,40))

norm = plt.Normalize(pearson_df_sort_pval["Pearson_p"].min(), pearson_df_sort_pval["Pearson_p"].max())
sm = plt.cm.ScalarMappable(cmap="rocket_r", norm=norm)

sns.barplot(pearson_df_sort_pval, x="Pearson_r", y="TFs",hue="Pearson_p", width=0.5, palette="rocket_r")


ax.set_ylabel('TFBS')

ax.xaxis.tick_top()
ax.set_xlabel('Pearson Correlation')
ax.xaxis.set_label_position("top")

ax.xaxis.grid(True)
ax.get_legend().remove()

plt.subplots_adjust(top=0.9)
cbar = ax.figure.colorbar(sm, ax=ax, orientation= "horizontal", location="top")
cbar.set_label("-log10(p-value)")

#fig.savefig("/sybig/projects/GeneRegulation/data/jme/Bachelorarbeit/data/Single_TFBS/Boxplot/strand_ratio_Pearson_corr.png")


NameError: name 'plt' is not defined

### Filter for all nT (1) and all T (-1) VS. homotypic count

In [15]:
filtered_all_nT = flat_strand_df_red[flat_strand_df_red["homotypic_strand_orientation"]== 1]
filtered_all_T = flat_strand_df_red[flat_strand_df_red["homotypic_strand_orientation"]== -1]

In [17]:
pearson_all_nT = get_pearson(filtered_all_nT)
pearson_all_T = get_pearson(filtered_all_T)

/sybig/home/jme/Bachelorarbeit/.venv/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


In [18]:
pearson_all_nT.to_csv("/sybig/projects/GeneRegulation/data/jme/Bachelorarbeit/data/Single_TFBS/all_nT_Pearson.csv", index=False)
pearson_all_T.to_csv("/sybig/projects/GeneRegulation/data/jme/Bachelorarbeit/data/Single_TFBS/all_T_Pearson.csv", index=False)

In [19]:
nT_pearson_sort_pval, nT_filterd_constant, nT_filtered_by_p = refine_pearson_df(pearson_all_nT)

84 were filtered because they have  a constant homotypic count.
76 were filtered because they have  a p-value < 0.05.


/tmp/ipykernel_2124244/129602368.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pearson_df_sort_pval["Pearson_p"] = -np.log10(pearson_df_sort_pval["Pearson_p"])


In [31]:
T_pearson_sort_pval, T_filterd_constant, T_filtered_by_p = refine_pearson_df(pearson_all_T)

81 were filtered because they have  a constant homotypic count.
62 were filtered because they have  a p-value < 0.05.


/tmp/ipykernel_2124244/129602368.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pearson_df_sort_pval["Pearson_p"] = -np.log10(pearson_df_sort_pval["Pearson_p"])


In [34]:
T_filtered_by_p.sort()
T_filtered_by_p


array(['ASCL1', 'ATF2', 'ATF3', 'ATF4', 'ATF7', 'BATF', 'BCL6', 'CDX2',
       'CREM', 'CTCF', 'DUX4', 'E2F4', 'EOMES', 'ERG', 'ESRRA', 'ETV1',
       'FOS', 'FOSL1', 'FOSL2', 'FOXA1', 'FOXA2', 'GATA2', 'GLIS1',
       'HAND2', 'HNF1A', 'JUNB', 'JUND', 'LEF1', 'MAFF', 'MEF2B', 'MEIS1',
       'MITF', 'MYCN', 'NFIC', 'NFIL3', 'NFKB1', 'NFKB2', 'NFYB', 'NR2C2',
       'ONECUT2', 'PDX1', 'POU5F1', 'RBPJ', 'REST', 'RXRA', 'SIX1',
       'SOX13', 'SPIB', 'SREBF1', 'STAT1', 'STAT5A', 'TBX21', 'TBX5',
       'TCF12', 'TCF3', 'TCF7', 'TEAD1', 'TFAP2C', 'TP73', 'YY1', 'YY2',
       'ZNF143'], dtype=object)

#### Plot

In [ ]:
fig, ax = plt.subplots(figsize=(10,23))

norm = plt.Normalize(T_pearson_sort_pval["Pearson_p"].min(), T_pearson_sort_pval["Pearson_p"].max())
sm = plt.cm.ScalarMappable(cmap="rocket_r", norm=norm)

sns.barplot(T_pearson_sort_pval, x="Pearson_r", y="TFs",hue="Pearson_p", width=0.5, palette="rocket_r")


ax.set_ylabel('TFBS')

ax.xaxis.tick_top()
ax.set_xlabel('Pearson Correlation')
ax.xaxis.set_label_position("top")

ax.xaxis.grid(True)
ax.get_legend().remove()

plt.subplots_adjust(top=0.9)
cbar = ax.figure.colorbar(sm, ax=ax, orientation= "horizontal", location="top", )
cbar.set_label("-log10(p-value)")

#fig.savefig("/sybig/projects/GeneRegulation/data/jme/Bachelorarbeit/data/Single_TFBS/Boxplot/T_Pearson_homoCount_Expr.png")

In [46]:
# Table
print(T_pearson_sort_pval[T_pearson_sort_pval["Pearson_r"] > 0.05])

        TFs  Pearson_r  Pearson_p
235  POU2F2   0.125779  81.406493
47    SOX17   0.102589  21.679886
136   PRDM1   0.085749  26.487226
35   HOXB13   0.079833  35.760394
170   NR2C1   0.076655  24.890651
175  TWIST1   0.073217  33.137693
28     NFE2   0.072078  45.840222
173    PBX2   0.070202  69.124495
216   RUNX2   0.064363  60.067120
60     KLF1   0.059941  64.131997
148   CEBPG   0.058541  21.658866
245    TCF4   0.053581  28.592657
137   GRHL2   0.052059  32.844346
152   FOXP1   0.050801  20.285796
195   MYOD1   0.050293  21.819514


### All Tissues

In [9]:
def get_pearson_all_tissues(df, columns="homotypic_count", expr_start_iloc=9, expr_end_iloc=None):
    # default argument for expr_end_iloc
    if expr_end_iloc is None:
        expr_end_iloc = len(df.columns)
    


    tfs = df["tf"].unique()
    tissues = np.array(df.columns[expr_start_iloc:expr_end_iloc])
    
    # initialize Array with shape (0,54), This is important to vstack the 1D-Arrays for every tf, to genereate a 2D-array.
    r_all_tf = np.zeros([0,len(tissues)])
    p_all_tf = np.zeros([0,len(tissues)])

    tfs_lst = []
    for tf in tfs:
        df_tf = df[df["tf"] == tf]
        if len(df_tf) > 1:
            tfs_lst.append(tf)

            tfbs_counts = np.array(df_tf["homotypic_count"])
            expr = np.array(df_tf.iloc[:,expr_start_iloc:expr_end_iloc])

            r_all_tissue = np.array([])
            p_all_tissue = np.array([])
            for tissue in range(len(tissues)):
                tissue_expr = expr[:,tissue]

                r, p = pearsonr(tfbs_counts, tissue_expr)
                r_all_tissue = np.append(r_all_tissue, r)
                p_all_tissue = np.append(p_all_tissue, p)
            # Appand r and p for single tfs into one big 2d array
            r_all_tf = np.vstack((r_all_tf, r_all_tissue))
            p_all_tf = np.vstack((p_all_tf, p_all_tissue))

        else:
            print(f"{tf} has only 1 entry. No Correlation could be calculated")
        # Add columns to DataFrame and set index to tf names
        r_df = pd.DataFrame(r_all_tf, columns=tissues)
        r_df.index = tfs_lst
        p_df = pd.DataFrame(p_all_tf, columns=tissues)
        p_df.index = tfs_lst       
        
    return r_df, p_df

In [ ]:
r_df, p_df = get_pearson_all_tissues(df_new, expr_start_iloc=10)

In [49]:
## Which TFBS have only one TFBS per Promotor?
# r_df[np.all(r_df.isna(), axis=1)].index.to_numpy()

# # Sort both DataFrames based on the sum of the correlation coefficient and remove NANs.
# r_df_s = r_df.reindex(abs(r_df.sum(axis=1)).sort_values(ascending=False).index).dropna()
# p_df_s = p_df.reindex(abs(r_df.sum(axis=1)).sort_values(ascending=False).index).dropna()


# r_df_filtered_by_p_value = r_df_s[p_df_s<0.05]


# r_df_filtered_by_p_value_remove_nan_rows = r_df_filtered_by_p_value[~np.all(r_df_filtered_by_p_value.isna(), axis=1)]
# r_df_filtered_by_p_value_remove_nan_rows_sort_by_mean = r_df_filtered_by_p_value_remove_nan_rows.reindex(abs(r_df_filtered_by_p_value_remove_nan_rows.mean(axis=1, skipna=True)).sort_values(ascending=False).index)

56

In [ ]:
value_max = max(max(abs(r_df_filtered_by_p_value.max())), max(abs(r_df_filtered_by_p_value.min())))
value_min = -value_max


fig, ax = plt.subplots(figsize=(10, 35))
heatmap = sns.heatmap(r_df_filtered_by_p_value_remove_nan_rows_sort_by_mean, cmap='coolwarm', fmt=".2f", linewidths=0.01,vmin=value_min, vmax=value_max, cbar_kws={"orientation": "horizontal", "location":"top", "label":"Pearson coefficient", "pad":0.09})
#plt.title('Genexpression in different tissues')
plt.xlabel('TFBS')
plt.ylabel('Tissues')

ax.xaxis.tick_top()
ax.set_xlabel("")
ax.xaxis.set_label_position("top")
plt.xticks(rotation=90)


plt.tight_layout()
#fig.savefig("/sybig/projects/GeneRegulation/data/jme/Bachelorarbeit/data/Single_TFBS/Heatmap/Pearson_filtered_remove_p_sort_by_mean_withoutNAN.png")
plt.show()